In [1]:
import pandas as pd
import os
import datetime
sys.path.append('C:\\Users\\trueblood\\projects\\pyce_tools')
import pyce_tools.pyce_tools as pt

ice to meet you.


Introduction to the tutorial.

In [ ]:
# Pre-processing of INP Data

In [ ]:
# Raw Data Calculation

In [4]:
# Create blank file
issues = None
type_ = 'aerosol'
process = 'uf'
collection_date = '06042020 '
sample_name = 'Bubbler 3 stage BLANK pc'
analysis_date = '01/01/21'
num_tubes = 26
vol_tube = 0.2
rinse_vol = 20
size = 'sub'
location = 'bubbler'
raw = pt.calculate_raw_blank(type_, process, location, sample_name, collection_date,
                  analysis_date, issues, num_tubes, vol_tube, rinse_vol, size)

...Raw blank data calculated!


In [6]:
# Calculate aerosol sample
issues = None
blank_source = '..\\data\\interim\\IN\\calculated\\blank\\bubbler_blank_uf_sub_060420_calculated.xlsx'
type_ = 'aerosol'
size = 'sub'
location = 'bubbler'
process = 'uf'
sample_name = 'Day 01 bubbler 3 stage PC'
collection_date = '17032020 11h25'
sample_stop_time = '18032020 09h45'
analysis_date = '01/01/2021'
num_tubes = 26
vol_tube = 0.2
flow_start = 10.50
flow_stop = 9.35
rinse_vol = 20

raw = pt.calculate_raw(blank_source, type_, location, process, sample_name, collection_date, analysis_date, issues, num_tubes, vol_tube, rinse_vol, size, flow_start, flow_stop, sample_stop_time)

...IN data calculated!
Calculated report file saved to ..\data\interim\IN\calculated\aerosol\bubbler\aerosol_bubbler_uf_sub_170320_1125_calculated.xlsx.


Running the above code will create a calculated report file in which the blank corrected values are given. Take a minute to open the report file now (file path is given above) and have a look yourself. Also be sure to check the tubeholderfilling tab which gives an overview of all the cells. You should compare this with the final image from the LINDA to ensure the automated process correctly identified frozen vs nonfrozen cells.

Also note a separate tab for heated vs unheated samples is given (e.g., summary_UF_UH and summary_UF_H). The code is written to assume the bottom 26 cells are unheated and the top 26 cells are heated. You can alter this as needed. Each summary tab has calculations for blank corrected concentrations and uncertainty as well as metadata.

In [ ]:
# Create Clean Data

Cleaning the available calculated report files is as sample as calling clean_calculated_in() function and passing the parameters for INP type and location. The script will automatically find all relevant files in the folder and combine them into a single time series.

In [12]:
# Create cleaned files of INP and uncertainty
pt.clean_calculated_in('aerosol','bubbler')
pt.calculate_wilson_errors('tutorial','bubbler','aerosol')

In [19]:
# Load Cleaned Data

Load the cleaned INP data and uncertainty files. Do a final bit of preprocessing. This is unique to each experiment but give an example below.

In [13]:
# Load cleaned data and create time timeseries index in New Zealand Standard Time
inp_bubbler = pd.read_csv('..\\data\\interim\\IN\\cleaned\\combinedtimeseries\\aerosol\\bubbler_17032020_17032020.csv', index_col='datetime')
inp_bubbler.index = pd.to_datetime(inp_bubbler.index, format='%d%m%Y %Hh%M')
inp_bubbler= inp_bubbler.tz_localize('NZ')

bubbler_error = pd.read_csv('..\\data\\interim\\IN\\cleaned\\combinedtimeseries\\aerosol\\bubbler_170320_170320wilson_error.csv', index_col='datetime', parse_dates=True)
bubbler_error.index=pd.to_datetime(bubbler_error.index, dayfirst=True, format='%d%m%Y %Hh%M')
bubbler_error= bubbler_error.tz_localize('NZ')


In [14]:
inp_bubbler.head()

,-18.0,-17.9,-17.8,-17.7,-17.6,-17.5,-17.4,-17.3,-17.2,-17.1,...,-1.2,-1.1,-1.0,start_date,stop_date,size,process,type,location,filtered
datetime,,,,,,,,,,,,,,,,,,,,,
2020-03-17 11:25:00+13:00,0.007185,0.005814,0.005212,0.003651,0.003651,0.003651,0.003195,0.003195,0.003195,0.003195,...,0.0,0.0,0.0,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf
2020-03-17 11:25:00+13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf


In [15]:
# Create melted dataframes with INP/m^3
inp_bubbler_melt=inp_bubbler.reset_index().melt(id_vars = ['datetime','start_date','stop_date','size','process', 'type','location','filtered'], var_name = 'temp', value_name='inp/l')
inp_bubbler_melt['inp/m^3'] = inp_bubbler_melt['inp/l']*1000

bubbler_error_melt = pd.melt(bubbler_error.reset_index(), id_vars = ['datetime','time','process','filtered','location','type','value_name', 'size'], var_name = 'temp', value_name='error (inp/l)')
bubbler_error_melt=bubbler_error_melt.drop(columns='time')
bubbler_error_melt['error (inp/m^3)'] = bubbler_error_melt['error (inp/l)']*1000

In [16]:
inp_bubbler_melt

,datetime,start_date,stop_date,size,process,type,location,filtered,temp,inp/l,inp/m^3
0,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-18.0,0.007185,7.184567
1,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-18.0,NaN,NaN
2,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-17.9,0.005814,5.813676
3,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-17.9,NaN,NaN
4,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-17.8,0.005212,5.211829
...,...,...,...,...,...,...,...,...,...,...,...
337,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-1.2,0.000000,0.000000
338,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-1.1,0.000000,0.000000
339,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-1.1,0.000000,0.000000
340,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-1.0,0.000000,0.000000


In [17]:
# create a pivoted uway error dataframe then merge with the actual data
bubbler_error_pivot = bubbler_error_melt.pivot_table(index=['process','datetime','temp','type','location','filtered'], values=['error (inp/l)'], columns='value_name')
bubbler_df = pd.merge(inp_bubbler_melt,bubbler_error_pivot.reset_index(), on=['datetime','temp','process','type','location','filtered'])

# clean column names
bubbler_df = bubbler_df.rename(columns={(
    'error (inp/l)','IN/L_lower'):'IN/L_lower',
    ('error (inp/l)','IN/L_upper'):'IN/L_upper',
    ('error (inp/l)','error_minus_y'):'error_minus_y',
    ('error (inp/l)','error_y'):'error_y'})

bubbler_df.set_index('datetime', inplace=True)

In [18]:
bubbler_df

,start_date,stop_date,size,process,type,location,filtered,temp,inp/l,inp/m^3,IN/L_lower,IN/L_upper,error_minus_y,error_y
datetime,,,,,,,,,,,,,,
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-18.0,0.007185,7.184567,0.004165,0.011240,0.003019,0.004055
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-18.0,NaN,NaN,0.015414,NaN,NaN,NaN
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-17.9,0.005814,5.813676,0.003292,0.009371,0.002521,0.003558
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-17.9,NaN,NaN,0.015414,NaN,NaN,NaN
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-17.8,0.005212,5.211829,0.002906,0.008553,0.002305,0.003342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-1.2,0.000000,0.000000,0.000000,0.001036,0.000000,0.001036
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-1.1,0.000000,0.000000,0.000000,0.001036,0.000000,0.001036
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-1.1,0.000000,0.000000,0.000000,0.001036,0.000000,0.001036


In [ ]:
# Calculated Surface Area Normalized INP Concentrations

In [ ]:
# Prepare SMPS Data

In [ ]:
# load scanotron data that was cleaned using pt.clean_inverted in the temp_control_cn.ipynb
dates = '2020-03-14_2020-03-27'
instr = 'scanotron' 

dN, dNdLogDp_full = pt.load_scano_data(dates, instr)

# this code requires a dNdLogDp and a dictionary of times
# create dictionary of times for each day that the INP bubbler was running, named in_time_dict
in_time_dict = {    
    '17': ['2020-03-17 11:25','2020-03-18 09:45'],
    }
# create empty smps dictionary
smps_dict = {}

# loop through each key in the in_time_dict and index those times from the dNdlogDp_full dataframe and save each of them as a dictionary in the smps dictionary
for day in in_time_dict:
    smps_dict[day] = dNdLogDp_full.loc[in_time_dict[day][0]:in_time_dict[day][1]]

#  remove time during cleaning
smps_dict['23']=smps_dict['23'][(smps_dict['23'].index <'2020-03-23 12:51:43') | (smps_dict['23'].index >'2020-03-23 15:22:14')]

# loop through the dictionary and remove massive outliers
from scipy import stats
for day in in_time_dict:
    z = abs(stats.zscore(smps_dict[day].sum(axis=1)))
    smps_dict[day] = smps_dict[day][(z<1)]

# create a daily standard deviation from the smps dictionary
smps_daily_std = {}
smps_daily_mean = {}

# create a daily mean dictionary from the smps dictionary
for day in smps_dict:
    smps_daily_mean[day] = smps_dict[day].mean()
    smps_daily_std[day] = smps_dict[day].std()

# turn them into dataframes
smps_daily_mean_df = pd.DataFrame(smps_daily_mean)
smps_daily_std_df = pd.DataFrame(smps_daily_std)

# clean it up a bit
smps_daily_mean_df.reset_index(inplace=True)
smps_daily_mean_df=smps_daily_mean_df.rename(columns={'index':'Dp'})
smps_daily_mean_df.set_index('Dp', inplace=True)

smps_daily_std_df.reset_index(inplace=True)
smps_daily_std_df=smps_daily_std_df.rename(columns={'index':'Dp'})
smps_daily_std_df.set_index('Dp', inplace=True)
smps_daily_mean_df.index=smps_daily_mean_df.index.astype(int)
smps_daily_std_df.index=smps_daily_std_df.index.astype(int)

In [ ]:
dAdLogDp, dA_total, dN_total, dAdLogDp_std = pt.surface_area(smps_daily_mean_df,smps_daily_std_df, 25)

In [ ]:
fig3=pt.plot_number_dist(smps_daily_mean_df.drop(columns=['Dp (m)', 'Dp2 (m2)', 'factorA']), smps_daily_std_df.drop(columns=['Dp (m)', 'Dp2 (m2)', 'factorA']))
fig4=pt.plot_surface_dist(dAdLogDp, dAdLogDp_std)

In [ ]:
fig3.update_layout(height=800, width=900)

In [ ]:
fig4.update_layout(height=800)

In [ ]:
# Normalize INP Data 

In [ ]:
inp_bubbler = pt.inp(inp_type='aerosol', inp_location='bubbler', cyto_location='n/a', cyto_data=None, uway_bio_data=None, inp_data=bubbler_df)
inp_bubbler.sa_normalize(dA_total)

In [ ]:
fig2=inp_bubbler.plot_ins_inp()
fig2.update_yaxes(type='log', title='INP per cm<sup>2</sup> of SSA Surface (D<sub>p</sub> = 10-5000nm)', exponentformat='power')
# fig.write_image("..\\reports\\figures\\10112020\\ssa_inp_comparison.png", scale=2)

In [ ]:
# Calculate Correlations

In [ ]:
inp_bubbler.inp.set_index('datetime',inplace=True)
inp_bubbler.correlations(temps=temps,process='UH', inp_units='inp_sa_normalized', size='sub')
inp_bubbler.correlations(temps=temps,process='H', inp_units='inp_sa_normalized', size='sub')

In [ ]:
temp = '-17.5'
inp_bubbler.results['UH'][temp]['corrs'].sort_values(by='p', ascending=True)[:10].style.set_caption(f"Correlations with Unheated Bubbler INP {temp}")